## Data Loading
เตรียม Dataset โดยแบ่งเป็น 8 files แล้วค่อย concat เป็น DataFrame เดียว เพื่อไม่ให้ Out of Memory

In [1]:
import pandas as pd
import glob
import os

# current_dir = os.path.dirname(os.path.abspath(__file__))
# project_root = os.path.abspath(os.path.join(current_dir, '..', '..'))
# input_path = os.path.join(project_root, 'dataset', 'bangkok_traffy.csv')
# # print(f"Reading data from: {input_path}

# OUTPUT_NAME = "num_province.csv"
# OUTPUT_CSV = os.path.join(project_root, 'data-insight', OUTPUT_NAME)

# 1. สร้าง list ของไฟล์ทั้งหมด (ใช้ glob)
file_list = sorted(glob.glob("./dataset/bangkok_traffy_part_*.csv"))

# 2. อ่านทีละไฟล์แล้ว append
df_list = []
for file in file_list:
    print(f"Reading {file} ...")
    df_list.append(pd.read_csv(file))

# 3. รวมทั้งหมดเป็น DataFrame เดียว
df = pd.concat(df_list, ignore_index=True)

# 4. ตรวจสอบจำนวนแถว
print(f"Total rows: {len(df)}")

Reading ./dataset\bangkok_traffy_part_1.csv ...
Reading ./dataset\bangkok_traffy_part_2.csv ...
Reading ./dataset\bangkok_traffy_part_3.csv ...
Reading ./dataset\bangkok_traffy_part_4.csv ...
Reading ./dataset\bangkok_traffy_part_5.csv ...
Reading ./dataset\bangkok_traffy_part_6.csv ...
Reading ./dataset\bangkok_traffy_part_7.csv ...
Reading ./dataset\bangkok_traffy_part_8.csv ...
Total rows: 787026


# เช็คค่าต่างๆจาก DataFrame

**เช็ค จำนวน records**

In [2]:
print(len(df))

787026


**เช็ค province ที่มีทั้งหมด**

In [3]:
print(df['province'].unique())

['กรุงเทพมหานคร' 'ศรีสะเกษ' 'นนทบุรี' 'สมุทรปราการ' 'จังหวัดกรุงเทพมหานคร'
 'สมุทรสาคร' 'ปทุมธานี' 'นครปฐม' 'สุพรรณบุรี' 'เชียงราย' nan 'จังหวัดLac'
 'จังหวัดนครปฐม' 'จังหวัดSahel' 'จังหวัดสมุทรสาคร' 'จังหวัดชลบุรี'
 'จังหวัดBorno' 'จังหวัดTillabéri' 'จังหวัดจังหวัด กรุงเทพมหานคร'
 'จังหวัดสมุทรปราการ' 'จังหวัดปทุมธานี' 'จังหวัดนนทบุรี'
 'จังหวัดพระนครศรีอยุธยา' 'จังหวัดนครศรีธรรมราช' 'จังหวัดฉะเชิงเทรา'
 'จังหวัดชุมพร' 'จังหวัดPerak' 'จังหวัดRiau' 'ชลบุรี' 'จังหวัดBangkok'
 'จังหวัดจังหวัดกรุงเทพมหานคร' 'จังหวัดนครสวรรค์' 'จังหวัดสงขลา' 'ลำพูน'
 'จังหวัดจ.ฉะเชิงเทรา' 'พิษณุโลก' 'ภูเก็ต' 'นครราชสีมา' 'จังหวัดสระบุรี'
 'ราชบุรี' 'สงขลา' 'จังหวัดประจวบคีรีขันธ์' 'สุราษฎร์ธานี' 'จังหวัดลพบุรี'
 'เชียงใหม่' 'ยโสธร' 'กาญจนบุรี' 'พังงา' 'อุดรธานี' 'จังหวัดราชบุรี'
 'จังหวัดเพชรบุรี' 'สุรินทร์' 'นครสวรรค์' 'ลพบุรี' 'จังหวัดจังหวัดพะเยา'
 'จังหวัดเชียงใหม่' 'จังหวัดจังหวัด' 'บุรีรัมย์' 'พระนครศรีอยุธยา'
 'จังหวัดจังหวัดพระนครศรีอยุธยา' 'จังหวัดกรุงเทพฯ' 'แพร่' 'ขอนแก่น'
 'จังหวัดจ.สุพรรณบุรี' 

**เช็ค language encoding ของ province**

In [ ]:
import chardet   #Optional
 
with open('./dataset/bangkok_traffy_100.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))
print(result)


FileNotFoundError: [Errno 2] No such file or directory: './dataset/bangkok_traffy_100.csv'

# Data Cleaning & Quality Assurance

**เคลียร์ : ข้อมูลที่ไม่ได้อยู่ในกรุงเทพ**

In [10]:
import pandas as pd

# normalize: lowercase + strip
df['province_clean'] = (
    df['province']
    .astype(str)
    .str.lower()
    .str.replace(" ", "")  # ลบช่องว่างเพื่อ match ได้กว้างขึ้น
    .str.strip()
)

# list ของ pattern ที่ต้อง match (กรุงเทพทุกรูปแบบ)
bangkok_patterns = ["bangkok", "กรุงเทพ"]

# filter row ที่ตรงกับ pattern
df_bangkok = df[df['province_clean'].str.contains('|'.join(bangkok_patterns), na=False)].copy()

# 👉 เซต province ให้เป็น "กรุงเทพมหานคร" ทั้งหมด
df_bangkok['province'] = "กรุงเทพมหานคร"

print(f"จำนวนแถว Bangkok: {len(df_bangkok)}")
print(df_bangkok['province'].unique())

# ลบ column ชั่วคราว
df_bangkok.drop(columns=['province_clean'], inplace=True)


จำนวนแถว Bangkok: 785662
['กรุงเทพมหานคร']


**เคลียร์ column ที่ไม่ได้ใช้**

In [11]:
# 1. ระบุชื่อ column ที่ต้องการลบ
columns_to_drop = ['comment', 'photo', 'photo_after', 'address', 'count_reopen']  # แก้เป็นชื่อ column ที่ไม่ต้องการ

# 2. ลบ column เหล่านั้น
df_clean = df_bangkok.drop(columns=columns_to_drop)

# 3. ตรวจสอบผลลัพธ์
print(df_clean.head())
print(df_clean.columns)

# 4. บันทึกไฟล์ใหม่ (ถ้าต้องการ)
# df_clean.to_csv("dataset/bangkok_traffy_clean.csv", index=False)


     ticket_id                 type                      organization  \
0  2021-FYJTFP          {ความสะอาด}                        เขตบางซื่อ   
1  2021-CGPMUN  {น้ำท่วม,ร้องเรียน}      เขตประเวศ,ฝ่ายโยธา เขตประเวศ   
2  2021-7XATFA              {สะพาน}                           เขตสาทร   
3  2021-9U2NJT            {น้ำท่วม}    เขตบางซื่อ,ฝ่ายโยธา เขตบางซื่อ   
4  2021-DVEWYM        {น้ำท่วม,ถนน}  เขตลาดพร้าว,ฝ่ายโยธา เขตลาดพร้าว   

               coords subdistrict  district       province  \
0  100.53084,13.81865         NaN       NaN  กรุงเทพมหานคร   
1  100.66709,13.67891     หนองบอน    ประเวศ  กรุงเทพมหานคร   
2  100.52649,13.72060     ยานนาวา      สาทร  กรุงเทพมหานคร   
3  100.53099,13.81853         NaN       NaN  กรุงเทพมหานคร   
4  100.59165,13.82280    ลาดพร้าว  ลาดพร้าว  กรุงเทพมหานคร   

                       timestamp      state  star  \
0  2021-09-03 12:51:09.453003+00  เสร็จสิ้น   NaN   
1  2021-09-19 14:56:08.924992+00  เสร็จสิ้น   4.0   
2  2021-09-26 05:03:52.594898

**เคลียร์ และ จัดโครงสร้าง Timestamp and Last Actitity ใหม่**

In [12]:
for col in ["timestamp", "last_activity"]:
    # แปลงเป็น datetime, UTC โดยให้ pandas infer format เอง
    df_clean[col] = pd.to_datetime(df_clean[col], utc=True, errors="coerce")
    
    # แปลง timezone เป็น Asia/Bangkok
    df_clean[col] = df_clean[col].dt.tz_convert("Asia/Bangkok")
    
    # เอา microseconds และวินาทีออก → เหลือ Year-Month-Day Hour:Minute
    df_clean[col] = df_clean[col].dt.strftime("%Y-%m-%d %H:%M")

# บันทึก CSV ใหม่
# df_clean.to_csv("./dataset/bangkok_traffy_clean.csv", index=False)

# print("Cleaned CSV saved with timestamp and last_activity formatted.")


**เพิ่ม time ของเวลาในการจัดการในแต่ละ task**

In [14]:
# แปลงคอลัมน์เวลาจาก string → datetime
df_clean['timestamp'] = pd.to_datetime(df_clean['timestamp'])
df_clean['last_activity'] = pd.to_datetime(df_clean['last_activity'])

# ตั้งค่า default = NaN
df_clean['time_to_complete'] = None

# เฉพาะแถวที่ state = "เสร็จสิ้น"
mask = df_clean['state'] == "เสร็จสิ้น"

# คำนวณจำนวนวัน
df_clean.loc[mask, 'time_to_complete'] = (
    (df_clean.loc[mask, 'last_activity'] - df_clean.loc[mask, 'timestamp']).dt.days
)


In [ ]:
# df_clean.to_csv("./dataset/bangkok_traffy_clean.csv", index=False)
# print("Cleaned CSV saved with timestamp and last_activity formatted.")

Cleaned CSV saved with timestamp and last_activity formatted.


**เคลียร์ missing value any ของ type , distict , subdistict รวมถึงถ้า type เป็น '{}'**

In [17]:
print("จำนวน missing value ของ type = ", len(df_clean[df_clean['type'] == '{}']))
print("จำนวน rows = ", len(df_clean))

จำนวน missing value ของ type =  0
จำนวน rows =  670564


In [16]:
import numpy as np

# 1) แทนค่าที่เป็น '{}' ให้เป็น NaN
df_clean['type'] = df_clean['type'].replace("{}", np.nan)

# 2) แทนค่าที่เป็นสตริงว่าง "" หรือ " " ให้เป็น NaN สำหรับทุกคอลัมน์
cols_to_clean = ['type', 'district', 'subdistrict']
df_clean[cols_to_clean] = df_clean[cols_to_clean].replace(["", " ", "  "], np.nan)

# 3) ลบแถวที่มี missing ในคอลัมน์สำคัญ
df_clean = df_clean.dropna(subset=['type', 'district', 'subdistrict'])

# 4) reset index หลัง drop
df_clean = df_clean.reset_index(drop=True)

print("หลังลบ missing rows แล้วเหลือ =", len(df_clean))

หลังลบ missing rows แล้วเหลือ = 670564


In [ ]:
# Optional : 
# df_clean.to_csv("dataset/df_clean.csv", index=False)

**แบ่ง organization**

In [21]:
import re

def extract_organization_list(raw):
    if pd.isna(raw):
        return []

    # แยกด้วย comma
    parts = [p.strip() for p in str(raw).split(",") if p.strip()]

    # ลบ duplicates
    parts = list(dict.fromkeys(parts))

    return parts

def pick_main_organization(org_list):
    """เลือกองค์กรหลักตามกติกา:
       - ถ้ามีหลายรายการ → ตัด 'เขต...' ออก แล้วเลือกตัวแรกที่ไม่ใช่เขต
       - ถ้าเหลือแต่ 'เขต...' อย่างเดียว → เอา 'เขต...' เป็นหลัก
    """

    if not org_list:
        return None

    # pattern เขตxxx เช่น เขตบางซื่อ, เขตประเวศ
    district_pattern = r"^เขต"

    # แยกเป็น 2 กลุ่ม
    non_district = [o for o in org_list if not re.match(district_pattern, o)]
    district_only = [o for o in org_list if re.match(district_pattern, o)]

    if non_district:
        # เอาตัวแรกที่ไม่ใช่เขตเป็น organization หลัก
        return non_district[0]
    else:
        # ถ้ามีแต่เขต ให้ใช้เขตเป็นองค์กร
        return district_only[0] if district_only else None

def clean_organization_list(org_list):
    """ล้างรายการ โดยลบ 'เขตxxxx' ถ้ามีองค์กรอื่นร่วมอยู่"""
    if not org_list:
        return []

    district_pattern = r"^เขต"

    non_district = [o for o in org_list if not re.match(district_pattern, o)]
    district_only = [o for o in org_list if re.match(district_pattern, o)]

    # ถ้ามี non-district → return แค่ non-district
    if non_district:
        return non_district

    # ถ้าไม่มี → return district ทั้งหมด (ส่วนใหญ่ 1 อัน)
    return district_only

In [23]:
# ---------------------------
# APPLY เข้ากับ DataFrame
# ---------------------------
df_clean["organization_list_raw"] = df_clean["organization"].apply(extract_organization_list)
df_clean["organization"] = df_clean["organization_list_raw"].apply(pick_main_organization)
df_clean["organization_list"] = df_clean["organization_list_raw"].apply(clean_organization_list)

# ลบคอลัมน์ raw หากไม่ต้องการ
df_clean = df_clean.drop(columns=["organization_list_raw"])

# Export CSV
df_clean.to_csv("dataset/df_clean_organization.csv", index=False)

print("✔ Saved to dataset/df_clean_organization.csv")
df_clean.head()

✔ Saved to dataset/df_clean_organization.csv


,ticket_id,type,organization,coords,subdistrict,district,province,timestamp,state,star,last_activity,time_to_complete,organization_list
0,2021-CGPMUN,"{น้ำท่วม,ร้องเรียน}",ฝ่ายโยธา เขตประเวศ,"100.66709,13.67891",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-09-19 21:56:00,เสร็จสิ้น,4.0,2022-06-21 15:21:00,274.0,[ฝ่ายโยธา เขตประเวศ]
1,2021-7XATFA,{สะพาน},เขตสาทร,"100.52649,13.72060",ยานนาวา,สาทร,กรุงเทพมหานคร,2021-09-26 12:03:00,เสร็จสิ้น,NaN,2022-06-06 08:17:00,252.0,[เขตสาทร]
2,2021-DVEWYM,"{น้ำท่วม,ถนน}",ฝ่ายโยธา เขตลาดพร้าว,"100.59165,13.82280",ลาดพร้าว,ลาดพร้าว,กรุงเทพมหานคร,2021-12-09 19:29:00,เสร็จสิ้น,5.0,2022-08-12 14:18:00,245.0,[ฝ่ายโยธา เขตลาดพร้าว]
3,2021-8N9ZP8,{ความสะอาด},ฝ่ายเทศกิจ เขตประเวศ,"100.64690,13.67083",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-18 21:50:00,เสร็จสิ้น,2.0,2024-11-26 11:17:00,1073.0,[ฝ่ายเทศกิจ เขตประเวศ]
4,2021-8BTWZB,{ท่อระบายน้ำ},ฝ่ายโยธา เขตประเวศ,"100.65440,13.68158",หนองบอน,ประเวศ,กรุงเทพมหานคร,2021-12-22 17:15:00,เสร็จสิ้น,5.0,2022-06-20 20:12:00,180.0,[ฝ่ายโยธา เขตประเวศ]


**Import External Source**

In [ ]:
%run src/function/scraping_ex_pm2_5.py
%run src/function/scraping_org_from_overpass.py

===== BKK PM2.5 Daily (All Subdistricts, Fast) =====
ช่วงวันที่: 2023-01-01 → 2025-12-07
จำนวนแขวงจาก BKK_CENTROIDS: 180 จุด

ใช้แขวงที่มีพิกัดจริง: 180 จุด


📍 เริ่ม: แขวงคลองต้นไทร เขตคลองสาน (13.720458, 100.504528)
   - คลองต้นไทร: ดึงช่วง 2023-01-01 → 2023-01-30

📍 เริ่ม: แขวงคลองสาน เขตคลองสาน (13.730057, 100.508595)
   - คลองสาน: ดึงช่วง 2023-01-01 → 2023-01-30

📍 เริ่ม: แขวงบางลำภูล่าง เขตคลองสาน (13.714204, 100.504252)
   - บางลำภูล่าง: ดึงช่วง 2023-01-01 → 2023-01-30

📍 เริ่ม: แขวงสมเด็จเจ้าพระยา เขตคลองสาน (13.733617, 100.498445)
   - สมเด็จเจ้าพระยา: ดึงช่วง 2023-01-01 → 2023-01-30

📍 เริ่ม: แขวงทรายกองดิน เขตคลองสามวา (13.845216, 100.743481)
   - ทรายกองดิน: ดึงช่วง 2023-01-01 → 2023-01-30
   - บางลำภูล่าง: ดึงช่วง 2023-01-31 → 2023-03-01   - คลองสาน: ดึงช่วง 2023-01-31 → 2023-03-01

   - คลองต้นไทร: ดึงช่วง 2023-01-31 → 2023-03-01
   - ทรายกองดิน: ดึงช่วง 2023-01-31 → 2023-03-01
   - สมเด็จเจ้าพระยา: ดึงช่วง 2023-01-31 → 2023-03-01
   - ทรายกองดิน: ดึงช่วง 2023-03-02 → 202